# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234528463255                   -0.50    8.0
  2   -7.250196826464       -1.80       -1.40    1.0
  3   -7.251315453740       -2.95       -2.13    4.0
  4   -7.251293333665   +   -4.66       -2.30    4.0
  5   -7.251326331150       -4.48       -2.62    3.0
  6   -7.251337492332       -4.95       -3.07    2.0
  7   -7.251338763913       -5.90       -3.96    1.0
  8   -7.251338788524       -7.61       -4.05    4.0
  9   -7.251338797692       -8.04       -4.51    3.0
 10   -7.251338798246       -9.26       -4.72    1.0
 11   -7.251338798692       -9.35       -5.40    3.0
 12   -7.251338798704      -10.93       -5.91    2.0
 13   -7.251338798704      -12.16       -6.18    2.0
 14   -7.251338798705      -12.56       -7.03    3.0
 15   -7.251338798705      -14.21       -7.44    1.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06102665742929522
[ Info: Arnoldi iteration step 2: normres = 0.636791264017885
[ Info: Arnoldi iteration step 3: normres = 0.8380551743741705
[ Info: Arnoldi iteration step 4: normres = 0.2984183132360873
[ Info: Arnoldi iteration step 5: normres = 0.453277758144635
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.28e-02, 7.20e-02, 3.27e-01, 3.01e-01, 3.65e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3449779379495476
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (9.50e-03, 1.13e-01, 2.97e-01, 1.02e-01, 7.63e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0836348056623581
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.95e-04, 1.18e-02, 2.01e-02, 3.69e-02, 6.13e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11354898416693567
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.97e-05, 